# Unificación de base de datos VU del departamento Calidad Fabricación - FSI

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None  # default='warn'

import os
pd.options.display.max_columns = 60

# Lista elementos

In [2]:
elementos = pd.read_csv('elementos.csv')
incidentes = pd.read_csv('incidentes.csv')
# elementos = pd.read_excel('Lista defectos.xlsx', 'Elemento')
# incidentes = pd.read_excel('Lista defectos.xlsx', 'Incidente')

# CAPS
Cargamos los datos crudos desde el excel, descartamos las columnas innecesarias y cambiamos el nombre a las columnas según la necesidad. Además creamos dos tablas, una para VP y otra para VU.

In [5]:
qcap_path = "../AUDITORIAS/CAPS/1-  PUNTO QCAP.xlsm"
caps = pd.read_excel(qcap_path, "DATOS")
caps.drop(columns=['MES', 'SEM', 'Caja de velocidades', 'Fabrica', 'Titulo defecto', 'Comentario defecto', 'Elementos', 'Defecto 2', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24','Unnamed: 25','Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 'Taller'], inplace=True)
caps[['INC','ELE']] = caps['Codigo defecto'].str.split("-",expand=True)
caps.rename(columns={'Familia':'FLIA', 'Usina':'USINA','Tipo  vehiculo':'TIPO', 'Avance producto':'DPTO', 'Motor':'MOTOR', 'Trasmision':'TRANS', 'Version':'VERS', 'Num. de Identificacion':'PJI', 'Codigo defecto':'DEF', 'Demerito (pt)':'DEM'}, inplace=True)
# caps.head()
caps['AUDI'] = 'CAPS'
caps = caps[['AUDI','FECHA', 'USINA', 'FLIA', 'TIPO', 'PJI', 'ELE', 'INC', 'DEM', 'DPTO']]
caps.replace({'DPTO': 'TCP'}, 'PINTURA', inplace=True)
caps.replace({'DPTO': 'TCT'}, 'SOLDADURA', inplace=True)
caps = caps[caps['PJI'].notna()]
#caps['PJI'] = caps['PJI'].apply(lambda x: str(int(x)))

# CAPS VU

In [6]:
filtvu = (caps['USINA'] == 'VU')
caps_vu = caps[filtvu]
caps_vu_generic = caps_vu[['AUDI', 'FECHA', 'FLIA', 'TIPO', 'PJI', 'ELE', 'INC', 'DEM', 'DPTO']]

# PESD VU

In [7]:
pesd_vu_path = "../AUDITORIAS/PESD/VU/7-PESD VU.xlsm"
pesd_vu = pd.read_excel(pesd_vu_path, "Datos crudos")
pesd_vu.rename(columns={'Code Localisation':'LOC', 'DEPARTAMENTO':'DPTO', 'COD DEFECTO':'DEF','Codigo Detección':'INC', 'Codigo Elemento':'ELE','Date de déclaration':'FECHA', 'Démérite':'DEM', 'Famille':'FLIA','Type':'TIPO', 'PJI avec défaut':'PJI', 'Code UET imputée':'UET', 'Code regroupement de pilotage':'TIPODEF'}, inplace=True)

In [8]:
pesd_vu['AUDI'] = 'PESD'
pesd_vu['TIPO'] = 'HND'
# Falta DPTO
pesd_vu_generic = pesd_vu[['AUDI', 'FECHA', 'FLIA', 'TIPO', 'PJI', 'ELE', 'INC', 'LOC', 'DEM', 'TIPODEF', 'UET']]
pesd_vu_generic.head()

,AUDI,FECHA,FLIA,TIPO,PJI,ELE,INC,LOC,DEM,TIPODEF,UET
0,PESD,2020-01-16,XND,HND,655200340151,1820,4891,NaN,V2,MOP,621
1,PESD,2020-07-07,XND,HND,655202330058,3308,2330,NaN,V2,MOP,621
2,PESD,2020-01-15,XND,HND,655200330162,3610,4932,NaN,NaN,MOP,600
3,PESD,2020-01-08,XND,HND,655200330211,3650,4531,NaN,NaN,MOP,621
4,PESD,2020-02-27,XND,HND,655201410001,3904,2330,NaN,NaN,MOP,648


# SAVES VU
Cargamos los datos crudos desde el excel, descartamos las columnas innecesarias y cambiamos el nombre a las columnas.

In [11]:
saves_vu_path = "../AUDITORIAS/SAVES/VU/SAVES VU + Tablero Integrado.xlsm"
saves_vu = pd.read_excel(saves_vu_path, "Datos")
saves_vu.drop(columns=['Año', 'Mes', 'Semana','Defecto2', 'A', 'Fabrica', 'VALIDADOR', 'GEOMETRIA', 'COMENTARIO', 'DETECCION PINTURA', 'OPERADOR FUGA CSC/CSCD', 'PUESTO FUGA CHECKMAN', 'ZONA CHECKMAN', 'OPERADOR FUGA CHECKMAN', 'Retocador', 'UET', 'DETECCION MONTAJE', 'PUESTO FUGA CSC /CSCD', 'ZONA', 'PESD','DEFECTO', 'Titulo defecto', 'PRO-PIE', 'ORIGEN'], inplace=True)
saves_vu[['ELE','INC']] = saves_vu['Codigo'].str.split("-",expand=True)
saves_vu.rename(columns={'Día':'FECHA', 'FAMILIA':'FLIA', 'DEMERITO':'DEM','Codigo':'DEF', 'Tipo  vehiculo':'TIPO', 'Avance PIEZA':'PIEZA', 'Motor':'MOTOR', 'Caja de velocidades':'CVELO', 'Trasmision':'TRANS', 'Version':'VERS','Num. de Identificacion':'PJI', 'Numero de fabricacion':'NFAB', 'GRUPOS':'TIPODEF'}, inplace=True)
saves_vu['AUDI'] = 'SAVES'

In [12]:
saves_vu_generic = saves_vu[['AUDI', 'FECHA', 'FLIA', 'TIPO', 'PJI', 'ELE', 'INC', 'DEM', 'TIPODEF', 'DPTO']]
saves_vu_generic['TIPODEF'].replace({'ASPECTO': 'ASP', 'GEOMETRIA':'GMTR', 'MODO OPERATORIO':'MOP', 'FUNCIONAMIENTO':'FCIO', 'RUIDOS':'RDOS', 'DEGRADACION':'DGRC', 'FALTANTE':'FTES', 'NO CONFORME':'NCON'}, inplace=True)

In [13]:
saves_vu_generic.head()

,AUDI,FECHA,FLIA,TIPO,PJI,ELE,INC,DEM,TIPODEF,DPTO
0,SAVES,2020-01-27,XND,H60A,6.552004e+11,1270,4514,V2,ASP,PINTURA
1,SAVES,2020-01-29,XND,H60A,6.552005e+11,1510,4313,V2,DGRC,MONTAJE
2,SAVES,2020-01-29,XND,H60A,6.552005e+11,1053,4508,V2,ASP,SOLDADURA
3,SAVES,2020-01-30,XND,H60A,6.552005e+11,8311,5611,V2,RDOS,RUIDOS
4,SAVES,2020-01-31,XND,H60A,6.552005e+11,8311,5611,V2,RDOS,RUIDOS


# DPU VU
Cargamos los datos crudos desde el excel, descartamos las columnas innecesarias y cambiamos el nombre a las columnas.

In [18]:
dpu_vu_path = "../AUDITORIAS/DPU/VU/Datos Brutos VU - 2020.xlsx"
dpu_vu = pd.read_excel(dpu_vu_path, "Datos Brutos")
# dpu_vu.drop(columns=['SEMANA','MES', 'ELEMNTO Final', 'ELEMNTO Libellé', 'Libellé localisation', 'DECLARACION UET', 'DECLARACION Equ.', 'RETOQUE UET', 'INCIDENTE Final', 'DEFECTOS', 'INCIDENTE Libellé', 'UET IMPUTACION', 'IMPUTACION Equ.', 'QRQC N1'], inplace=True)
dpu_vu.rename(columns={'DÍA':'FECHA', 'Code regroupement de pilotage':'TIPODEF', 'IMPUTACION Ate.':'TALLER', 'IMPUTACIONUET':'UET', 'Dém.':'DEM', 'IMPUTACIONDép.':'DPTO', 'Famille':'FLIA','Type':'TIPO', 'ELEMENTO Init.':'ELE', 'INCIDENTE Init.':'INC', 'Code Localisation':'LOC', 'IN/OFF Usine':'IN/OFF USINA', 'IN/OFF Départ.':'IN/OFF DPTO'}, inplace=True)

In [1]:
dpu_vu = dpu_vu[['FECHA','PJI', 'FLIA', 'TIPO', 'ELE', 'INC', 'LOC', 'TIPODEF', 'DEM', 'DPTO', 'TALLER', 'UET', 'IN/OFF USINA', 'IN/OFF DPTO']]
dpu_vu.insert(0, 'AUDI', 'DPU')
dpu_vu['DPTO'].replace({'DCHAP':'SOLDADURA', 'DPINT':'PINTURA', 'DMONT':'MONTAJE', 'MONTVU':'MONTAJE', 'EMB':'EMBUTICION', 'DAT':'DIVD'}, inplace=True)
dpu_vu['AUDI'] = 'DPU'

NameError: name 'dpu_vu' is not defined

# Datos limpios

In [20]:
table_vu = pd.concat([caps_vu_generic, pesd_vu_generic, saves_vu_generic, dpu_vu], sort=True)
table_vu = pd.merge(table_vu, elementos, on='ELE', how='left')
table_vu = pd.merge(table_vu, incidentes, on='INC', how='left')

In [21]:
# table_vu['AUDI'].value_counts()
table_vu

,AUDI,DEM,DPTO,ELE,FECHA,FLIA,IN/OFF DPTO,IN/OFF USINA,INC,LOC,PJI,TALLER,TIPO,TIPODEF,UET,ELEMENTO,INCIDENTE
0,CAPS,P2,SOLDADURA,4120,2020-01-27 00:00:00,H60A,NaN,NaN,0180,NaN,655200530232,NaN,H60A,NaN,NaN,PISO AV. O CABINA,ELEMENTO DE CHAPA IMPIDE EL MONTAJE
1,CAPS,P2,PINTURA,4414,2020-01-27 00:00:00,H60A,NaN,NaN,1270,NaN,655200420197,NaN,H60A,NaN,NaN,PAÑO EXT DE PUERTA ARD,HUELLAS
2,CAPS,P2,PINTURA,4466,2020-01-27 00:00:00,H60A,NaN,NaN,1010,NaN,655200420197,NaN,H60A,NaN,NaN,DOBLADO MARCO DE PUERTA AVD.,MASTICO MAL POSICIONADO
3,CAPS,P2,PINTURA,4420,2020-01-27 00:00:00,H60A,NaN,NaN,1110,NaN,655200450172,NaN,H60A,NaN,NaN,CAJON DE PUERTA ARD.,DESCOLGON
4,CAPS,P2,PINTURA,4412,2020-01-27 00:00:00,H60A,NaN,NaN,1180,NaN,655200450172,NaN,H60A,NaN,NaN,PAÑO EXT DE PUERTA AVD,GRANO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273081,DPU,NaN,PINTURA,4852,2020-12-12 00:00:00,XND,IN,IN,1020,M60,655210210001,PTAL1,HND,ASP,482,PISO UTILITARIO (CONJUNTO),FALTA DE MASTICO
273082,DPU,V1,MONTAJE,5042,2020-12-12 00:00:00,XND,IN,IN,2120,NaN,655210210001,SMVU,HND,MOP,SVU,TRIANGULO EXT. EMB.MARCO DE PTA. ARD.,NO PLACA
273083,DPU,V1,MONTAJE,5444,2020-12-12 00:00:00,XND,IN,IN,4300,NaN,655210210001,SMVU,HND,NCON,SVU,MANDO INT. DE PUERTA AVD.,NO CONFORME
273084,DPU,NaN,SOLDADURA,5779,2020-12-12 00:00:00,XND,OFF,OFF,2120,NaN,655210210001,TOLE4,HND,MOP,659,BARRA I. DE LA BACA,NO PLACA


In [22]:
table_vu['DEFECTO'] = table_vu['ELEMENTO'] + ' ' + table_vu['INCIDENTE']
table_vu.to_excel("clean_vu.xlsx") 

# GQU

# DSTAR VP